In [3]:
from typing import List, Tuple
from datetime import timedelta

from moviepy import VideoFileClip, concatenate_videoclips
import os

In [4]:
genre_dict = \
    {
        "drama": {
            "setup_conflict": {
                "description": "Установление основного конфликта",
                "start_percent": 5.0,
                "end_percent": 15.0
            },
            "character_development": {
                "description": "Эмоциональное развитие персонажа",
                "start_percent": 20.0,
                "end_percent": 40.0
            },
            "plot_twist": {
                "description": "Поворот сюжета или неожиданное развитие",
                "start_percent": 50.0,
                "end_percent": 60.0
            },
            "climax_confrontation": {
                "description": "Главный конфликт между героями",
                "start_percent": 60.0,
                "end_percent": 75.0
            },
            "partial_resolution": {
                "description": "Частичное разрешение, но с продолжением",
                "start_percent": 80.0,
                "end_percent": 90.0
            },
            "cliffhanger": {
                "description": "Неожиданный поворот или намёк на следующую серию",
                "start_percent": 95.0,
                "end_percent": 100.0
            }
        },

        "crime": {
            "crime_discovery": {
                "description": "Обнаружение преступления или загадка",
                "start_percent": 0.0,
                "end_percent": 10.0
            },
            "gathering_clues": {
                "description": "Расследование, сбор улик",
                "start_percent": 15.0,
                "end_percent": 35.0
            },
            "false_lead": {
                "description": "Ложный след или неправильная версия",
                "start_percent": 35.0,
                "end_percent": 50.0
            },
            "truth_revealed": {
                "description": "Открытие истинного преступника или причины",
                "start_percent": 60.0,
                "end_percent": 75.0
            },
            "resolution": {
                "description": "Завершение дела или арест",
                "start_percent": 80.0,
                "end_percent": 90.0
            },
            "emotional_ending": {
                "description": "Реакция главных героев или намёк на будущее",
                "start_percent": 90.0,
                "end_percent": 100.0
            }
        },

        "fantasy": {
            "quest_announced": {
                "description": "Объявление цели или миссии",
                "start_percent": 0.0,
                "end_percent": 10.0
            },
            "trials": {
                "description": "Пройденные испытания или битвы",
                "start_percent": 10.0,
                "end_percent": 50.0
            },
            "betrayal_or_failure": {
                "description": "Предательство или крупная потеря",
                "start_percent": 50.0,
                "end_percent": 65.0
            },
            "power_or_knowledge_gained": {
                "description": "Герои получают силу или знание для победы",
                "start_percent": 65.0,
                "end_percent": 80.0
            },
            "final_battle": {
                "description": "Финальная битва с главным злом",
                "start_percent": 80.0,
                "end_percent": 95.0
            },
            "new_horizon": {
                "description": "Намёк на новое приключение или путь",
                "start_percent": 95.0,
                "end_percent": 100.0
            }
        },

        "comedy": {
            "funny_situation_introduced": {
                "description": "Введение забавной ситуации",
                "start_percent": 0.0,
                "end_percent": 10.0
            },
            "absurdity_increases": {
                "description": "Усиление абсурда или ошибок",
                "start_percent": 10.0,
                "end_percent": 30.0
            },
            "peak_of_humor": {
                "description": "Максимальный юмор, точка кульминации",
                "start_percent": 30.0,
                "end_percent": 50.0
            },
            "attempt_to_fix": {
                "description": "Попытка исправить ситуацию",
                "start_percent": 50.0,
                "end_percent": 70.0
            },
            "unexpected_result": {
                "description": "Неожиданный смешной результат",
                "start_percent": 70.0,
                "end_percent": 90.0
            },
            "ironic_conclusion": {
                "description": "Шутка на прощание или клиффхенгер",
                "start_percent": 90.0,
                "end_percent": 100.0
            }
        },

        "sci-fi": {
            "technology_introduced": {
                "description": "Введение нового явления или технологии",
                "start_percent": 0.0,
                "end_percent": 10.0
            },
            "consequences_explored": {
                "description": "Изучение последствий использования",
                "start_percent": 10.0,
                "end_percent": 40.0
            },
            "ethical_dilemma": {
                "description": "Моральный выбор или риск",
                "start_percent": 40.0,
                "end_percent": 60.0
            },
            "crisis_occurs": {
                "description": "Происходит кризис или угроза",
                "start_percent": 60.0,
                "end_percent": 80.0
            },
            "solution_found": {
                "description": "Находится решение или новое понимание",
                "start_percent": 80.0,
                "end_percent": 95.0
            },
            "philosophical_note": {
                "description": "Философский вывод или вопрос",
                "start_percent": 95.0,
                "end_percent": 100.0
            }
        },

        "horror": {
            "atmosphere_set": {
                "description": "Создание атмосферы страха",
                "start_percent": 0.0,
                "end_percent": 10.0
            },
            "first_fear": {
                "description": "Первый страх или сигнал опасности",
                "start_percent": 10.0,
                "end_percent": 25.0
            },
            "tension_builds": {
                "description": "Наращивание напряжения",
                "start_percent": 25.0,
                "end_percent": 50.0
            },
            "main_horror_moment": {
                "description": "Главный момент ужаса или жертва",
                "start_percent": 50.0,
                "end_percent": 70.0
            },
            "escape_or_chase": {
                "description": "Погоня или попытка спастись",
                "start_percent": 70.0,
                "end_percent": 85.0
            },
            "unresolved_end": {
                "description": "Неожиданный финал или намёк на продолжение",
                "start_percent": 85.0,
                "end_percent": 100.0
            }
        },
        "default": {
            "setup": {
                "description": "Введение основного конфликта или ситуации",
                "start_percent": 5.0,
                "end_percent": 15.0
            },
            "development": {
                "description": "Развитие событий, углубление в сюжет",
                "start_percent": 20.0,
                "end_percent": 45.0
            },
            "plot_twist": {
                "description": "Неожиданный поворот или осложнение",
                "start_percent": 50.0,
                "end_percent": 65.0
            },
            "climax": {
                "description": "Кульминационный момент действия",
                "start_percent": 70.0,
                "end_percent": 85.0
            },
            "cliffhanger": {
                "description": "Финальный намёк на следующую серию",
                "start_percent": 90.0,
                "end_percent": 100.0
            }
        }

    }

In [5]:
UNIFIED_KEY_MOMENTS = {
    "setup": {
        "description": "Введение основной ситуации или конфликта"
    },
    "development": {
        "description": "Развитие сюжета, углубление в историю"
    },
    "twist": {
        "description": "Неожиданный поворот или осложнение"
    },
    "climax": {
        "description": "Кульминационный момент"
    },
    "resolution": {
        "description": "Разрешение или намёк на продолжение"
    },
    "cliffhanger": {
        "description": "Финальное событие или клиффхенгер"
    }
}

In [6]:
def unify_moments(genre_data):
    mapping = {
        "setup": None,
        "development": None,
        "twist": None,
        "climax": None,
        "resolution": None,
        "cliffhanger": None
    }

    # Попытка найти соответствующее поле в жанре
    for key in genre_data:
        if "setup" in key or "introduced" in key or "discovery" in key or "announced" in key:
            mapping["setup"] = key
        elif "develop" in key or "trial" in key or "consequence" in key or "gathering" in key:
            mapping["development"] = key
        elif "twist" in key or "false" in key or "betrayal" in key or "ethical" in key:
            mapping["twist"] = key
        elif "climax" in key or "battle" in key or "final" in key or "truth" in key:
            mapping["climax"] = key
        elif "resolution" in key or "ending" in key or "solution" in key or "conclusion" in key:
            mapping["resolution"] = key
        elif "cliffhanger" in key or "new" in key or "hint" in key or "philosophical" in key:
            mapping["cliffhanger"] = key

    return mapping

In [7]:
# Получаем маппинг для жанра "fantasy"
mapping = unify_moments(genre_dict["fantasy"])

# Теперь можем получать унифицированные временные метки
unified = {}
for uk, genre_key in mapping.items():
    if genre_key:
        unified[uk] = genre_dict["fantasy"][genre_key]
    else:
        unified[uk] = None

# Теперь можно генерировать рекап по unified-ключам

In [8]:
def generate_recap(genre, total_seconds):
    unified_moments = {}

    # Выбираем жанр или default
    genre_data = genre_dict.get(genre.lower(), genre_dict["default"])
    mapping = unify_moments(genre_data)

    for unified_key, genre_key in mapping.items():
        if genre_key:
            event = genre_data[genre_key]
            start_time = round(total_seconds * event["start_percent"] / 100)
            end_time = round(total_seconds * event["end_percent"] / 100)
            unified_moments[unified_key] = {
                "description": event["description"],
                "start": str(timedelta(seconds=start_time)),
                "end": str(timedelta(seconds=end_time))
            }
        else:
            unified_moments[unified_key] = None

    return unified_moments

In [9]:
recap = generate_recap("default", 2500)
recap

{'setup': {'description': 'Введение основного конфликта или ситуации',
  'start': '0:02:05',
  'end': '0:06:15'},
 'development': {'description': 'Развитие событий, углубление в сюжет',
  'start': '0:08:20',
  'end': '0:18:45'},
 'twist': {'description': 'Неожиданный поворот или осложнение',
  'start': '0:20:50',
  'end': '0:27:05'},
 'climax': {'description': 'Кульминационный момент действия',
  'start': '0:29:10',
  'end': '0:35:25'},
 'resolution': None,
 'cliffhanger': {'description': 'Финальный намёк на следующую серию',
  'start': '0:37:30',
  'end': '0:41:40'}}

In [ ]:
def create_recap(video_path, key_moments, recap_duration=45, output_path="recap.mp4"):
    # Открываем исходное видео
    clip = VideoFileClip(video_path)

    # Получаем все возможные фрагменты
    clips = []
    for name, times in key_moments.items():
        if times and isinstance(times, dict) and 'start' in times and 'end' in times:
            start = times['start']
            end = times['end']
            if start < end:  # защита от нулевых/обратных интервалов
                clips.append(clip.subclip(start, end))

    # Объединяем фрагменты
    final_clip = concatenate_videoclips(clips)

    # Обрезаем до нужной длины
    if final_clip.duration > recap_duration:
        final_clip = final_clip.subclip(0, recap_duration)

    # Сохраняем итоговый рекап
    final_clip.write_videofile(output_path, codec="libx264", audio_codec="aac")

    print(f" Рекап создан и сохранён как {output_path}")
    return output_path